Let's admit it, bayesian modeling on time series is slow. In pymc3, it typically implies using theano `scan` operation. Here, we will show how to profile one step of the kalman filter, as well as the scan operation over the time series.

First, load the required packages:

In [1]:
import numpy              as np
import theano
import theano.tensor      as tt

import kalman

We will use the same data as in the `01_RandomWalkPlusObservation` notebook.

In [2]:
# True values
T = 500                 # Time steps
sigma2_eps0 = 3         # Variance of the observation noise
sigma2_eta0 = 10        # Variance in the update of the mean

# Simulate data
np.random.seed(12345)
eps = np.random.normal(scale=sigma2_eps0**0.5, size=T)
eta = np.random.normal(scale=sigma2_eta0**0.5, size=T)
mu = np.cumsum(eta)
y = mu + eps

Next, we create all the tensors required to describe our model:

In [3]:
# Upon using pymc3, the following theano configuration flag is changed,
# leading to tensors being required to have test values
#theano.config.compute_test_value = 'ignore'

# Position and uncertainty of the observation
ai = tt.dvector(name='ai')
Pi = tt.dmatrix(name='Pi')

# Tensors for the measurement equation
Z = tt.dmatrix(name='Z')
d = tt.dvector(name='d')
H = tt.dmatrix(name='H')

# Tensors for the transition equation
T = tt.dmatrix(name='T')
c = tt.dvector(name='c')
R = tt.dmatrix(name='R')
Q = tt.dmatrix(name='Q')

We will also create some actual values for them:

In [4]:
ɛ_σ2 = 3.
η_σ2 = 10.

args = dict(ai = np.array([0.]),
            Pi = np.array([[1e6]]),
            Z = np.array([[1.]]),
            d = np.array([0.]),
            H = np.array([[ɛ_σ2]]),
            T = np.array([[1.]]),
            c = np.array([0.]),
            R = np.array([[1.]]),
            Q = np.array([[η_σ2]]))

Let's calculate the likelihood of the observed values, given the parameters above:

In [5]:
argst = (y[:,None], ai, Pi, Z, d, H, T, c, R, Q)
(at, Pt, lliks), updates = kalman.KalmanFilter.filter(*argst)

f = theano.function([ai, Pi, Z, d, H, T, c, R, Q], lliks)

llik = f(**args)
llik[1:].sum()

-1369.7346722999789

Time required for the log-likelihood calculation:

In [6]:
print('Measuring time...')
%timeit f(**args)

Measuring time...
10.4 ms ± 42.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Profiling a non-scan operation is relatively simple. As an example, let's create a function to calculate the first time step of the Kalman filter:

In [7]:
Y0 = tt.dvector(name='Y0')
_,_,llik = kalman.KalmanFilter.oneStep(Y0, ai, Pi, Z, d, H, T, c, R, Q)

profiler = theano.compile.ScanProfileStats()
f = theano.function([Y0, ai, Pi, Z, d, H, T, c, R, Q], llik, profile=profiler)

f(y[0,None], **args);

profiler.summary()

Class
---
<% time> <sum %> <apply time> <time per call> <type> <#call> <#apply> <Class name>
  28.9%    28.9%       0.000s       3.52e-06s     C        4       4   theano.tensor.blas.Dot22
  28.9%    57.8%       0.000s       3.52e-06s     C        4       4   theano.tensor.blas_c.CGemv
  10.8%    68.6%       0.000s       1.05e-06s     C        5       5   theano.tensor.elemwise.DimShuffle
  10.3%    78.9%       0.000s       2.50e-06s     C        2       2   theano.tensor.blas.Gemm
   8.8%    87.7%       0.000s       1.43e-06s     C        3       3   theano.tensor.elemwise.Elemwise
   6.4%    94.1%       0.000s       1.55e-06s     C        2       2   theano.tensor.basic.AllocEmpty
   3.9%    98.0%       0.000s       1.91e-06s     C        1       1   theano.tensor.subtensor.Subtensor
   2.0%   100.0%       0.000s       4.77e-07s     C        2       2   theano.compile.ops.Shape_i
   ... (remaining 0 Classes account for   0.00%(0.00s) of the runtime)

Ops
---
<% time> <sum %> <apply t

Repeating the procedure with a scan procedure, we can see that the code inside it is not profiled. It took me a while to make it work (not even stackoverflow helped!!!). In the end, this is how I made it work:

In [8]:
Y0 = y[:,None]
profiler = theano.compile.ScanProfileStats()
(_,_,llik),_ = kalman.KalmanFilter.filter(Y0, ai, Pi, Z, d, H, T, c, R, Q, profile=profiler)

f = theano.function([ai, Pi, Z, d, H, T, c, R, Q], llik, profile=profiler)

f(**args);

# Select the node corresponding to the scan operation
scan_op = next(k for k in profiler.op_nodes()
                     if isinstance(k, theano.scan_module.scan_op.Scan))
scan_op.profile.summary()


Scan Op profiling
  Message: None
  Time in 1 calls of the op (for a total of 500 steps) 1.379228e-02s

  Total time spent in calling the VM 1.050663e-02s (76.178%)
  Total overhead (computing slices..) 3.285646e-03s (23.822%)

Class
---
<% time> <sum %> <apply time> <time per call> <type> <#call> <#apply> <Class name>
  33.3%    33.3%       0.002s       9.70e-07s     C     2500       5   theano.tensor.blas_c.CGemv
  26.4%    59.7%       0.002s       6.41e-07s     C     3000       6   theano.tensor.blas.Dot22
  11.4%    71.1%       0.001s       4.13e-07s     C     2000       4   theano.tensor.elemwise.Elemwise
  10.6%    81.7%       0.001s       7.74e-07s     C     1000       2   theano.tensor.blas.Gemm
   7.7%    89.4%       0.001s       5.57e-07s     C     1000       2   theano.tensor.elemwise.DimShuffle
   7.2%    96.6%       0.001s       1.05e-06s     C      500       1   theano.tensor.subtensor.Subtensor
   2.5%    99.2%       0.000s       1.82e-07s     C     1000       2   thean